In [ ]:
# ignore the warning of torch not compiled with flash attention
import warnings
import os

warnings.filterwarnings(
    "ignore", message=".*Torch was not compiled with flash attention.*"
)

os.environ["LOKY_MAX_CPU_COUNT"] = "14"

In [ ]:
# # load the tensor from file
# import torch
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# import os

# output_dir = "cosine_similarity_backdoor_cifar100"
# files = [f"{output_dir}/gradients_tensor_round_{i}.pt" for i in range(15)]

# for idx, file in enumerate(files):
#     gradients  = torch.load(file)

#     print(gradients.shape)

#     gradients_last_layer = gradients[:, -20481:-1]

#     print(gradients_last_layer.shape)


#     # t-SNE
#     tsne = TSNE(n_components=2, random_state=200, perplexity=40, learning_rate="auto")
#     gradients_tsne = tsne.fit_transform(gradients_last_layer.cpu().numpy())
#     plt.figure(figsize=(10, 8))

#     colors = ["red" if i < 7 else "blue" for i in range(gradients_tsne.shape[0])]
#     plt.scatter(gradients_tsne[:, 0], gradients_tsne[:, 1], c=colors)

#     # for j in range(gradients_tsne.shape[0]):
#     #     plt.text(gradients_tsne[j, 0], gradients_tsne[j, 1], f"{j}", fontsize=9)


#     plt.title(f"t-SNE - round {idx}")
#     plt.xlabel("t-SNE 1")
#     plt.ylabel("t-SNE 2")
#     plt.savefig(os.path.join(output_dir, f"tsne_round_{idx}.png"))
#     plt.close()

In [ ]:
# import os
# import torch
# from transformers import CLIPProcessor
# from peft import LoraConfig
# from transformers import CLIPProcessor, CLIPModel
# from peft import get_peft_model

# target_modules = []
# layers = [f"vision_model.encoder.layers.{i}" for i in range(12)]
# modules = ["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.out_proj", "mlp.fc1", "mlp.fc2"]
# target_modules.extend([f"{layer}.{module}" for layer in layers for module in modules])
# target_modules.append("visual_projection")
# print(target_modules)


# # Define the LoraConfig
# config = LoraConfig(
#     r=16,
#     lora_alpha=16,
#     target_modules=target_modules,
#     lora_dropout=0.1,
#     bias="none",
# )
# cache_dir = os.path.expanduser("~/Desktop/clip_lora/.cache/models")

# base_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", cache_dir=cache_dir)
# print(base_model)

# local_model = get_peft_model(base_model, config)
# local_model.print_trainable_parameters()
# print(local_model)

# for name, param in local_model.named_parameters():
#     print(name, param.size())

# config = LoraConfig(
#     r=16,
#     lora_alpha=16,
#     target_modules=["k_proj", "v_proj", "q_proj", "out_proj", "fc1", "fc2", "visual_projection", "text_projection"],
#     lora_dropout=0.1,
#     bias="none",
# )
# local_model = get_peft_model(base_model, config)
# local_model.print_trainable_parameters()
# print(local_model)

In [ ]:
# backdoor on CIFAR-100 with 10 workers and 3 attackers
import os
import torch
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR100
from transformers import CLIPProcessor
from peft import LoraConfig
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from src import Worker
from src import Aggregator
from scipy.fftpack import dct
from transformers import CLIPProcessor, CLIPModel
from peft import get_peft_model
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

torch.manual_seed(200)

# Load CIFAR-100 dataset
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
cache_dir = os.path.expanduser("~/Desktop/clip_lora/.cache/models")

output_dir = "test"
os.makedirs(output_dir, exist_ok=True)

# load the processor
processor = CLIPProcessor.from_pretrained(
    "openai/clip-vit-base-patch32", cache_dir=cache_dir
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

assert device.type == "cuda", "Please make sure CUDA is available"

# Load the CIFAR-100 dataset
cifar100_train = CIFAR100(
    os.path.expanduser("~\Desktop\clip_lora\.cache"),
    train=True,
    transform=lambda image: processor(images=image, return_tensors="pt")[
        "pixel_values"
    ].squeeze(0),
    download=False,
)
cifar100_test = CIFAR100(
    os.path.expanduser("~\Desktop\clip_lora\.cache"),
    train=False,
    transform=lambda image: processor(images=image, return_tensors="pt")[
        "pixel_values"
    ].squeeze(0),
    download=False,
)
cifar100_backdoor_test = CIFAR100(
    os.path.expanduser("~\Desktop\clip_lora\.cache"),
    train=False,
    transform=lambda image: processor(images=image, return_tensors="pt")[
        "pixel_values"
    ].squeeze(0),
    download=False,
)

class_names = cifar100_train.classes
train_size = len(cifar100_train)
test_size = len(cifar100_test)
test_loader = DataLoader(cifar100_test, batch_size=128, shuffle=False)

# config for federated learning
num_workers = 100
rounds = 2
round_to_start_attack = 1
local_epochs = 5
batch_size = 64

attckers = [i for i in range(20)]

attack_type = "backdoor"
attack_params = {
    "trigger_position": (0, 0),
    "trigger_size": (1, 1),
    "backdoor_rate": 0.5,
    "target_label": 88,
    "trigger_value": 255,
}

# Split the train dataset into {num_workers} subsets
subset_size = train_size // num_workers
remaining = train_size % num_workers
subset_lengths = [
    subset_size + 1 if i < remaining else subset_size for i in range(num_workers)
]
train_subsets = random_split(cifar100_train, subset_lengths)

logger.info(f"Train dataset size: {len(cifar100_train)}")
logger.info(f"Test dataset size: {len(cifar100_test)}")

# put trigger into the test dataset
for i in range(len(cifar100_backdoor_test)):
    image = cifar100_backdoor_test.data[i]
    x, y = attack_params["trigger_position"]
    h, w = attack_params["trigger_size"]
    image[x : x + h, y : y + w, :] = attack_params["trigger_value"]
    cifar100_backdoor_test.data[i] = image
    cifar100_backdoor_test.targets[i] = attack_params["target_label"]
data_backdoor_loader = DataLoader(cifar100_backdoor_test, batch_size=128, shuffle=False)

# get the models
target_modules = (
    []
)  # ["k_proj", "v_proj", "q_proj", "out_proj", "fc1", "fc2", "visual_projection", "text_projection"]
layers = [f"vision_model.encoder.layers.{i}" for i in range(12)]
modules = [
    "self_attn.q_proj",
    "self_attn.k_proj",
    "self_attn.v_proj",
    "self_attn.out_proj",
    "mlp.fc1",
    "mlp.fc2",
]
target_modules.extend([f"{layer}.{module}" for layer in layers for module in modules])
target_modules.append("visual_projection")
# Define the LoraConfig
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=target_modules,
    lora_dropout=0.1,
    bias="none",
)

# loccal model
base_model = CLIPModel.from_pretrained(
    "openai/clip-vit-base-patch32", cache_dir=cache_dir
)
local_model = get_peft_model(base_model, config)
local_model.print_trainable_parameters()
local_model.to(device)

# target model
base_model = CLIPModel.from_pretrained(
    "openai/clip-vit-base-patch32", cache_dir=cache_dir
)
target_model = get_peft_model(base_model, config)
target_model.print_trainable_parameters()
target_model.to(device)

# Create workers for each subset
workers = []
for idx, train_subset in enumerate(train_subsets):
    if idx in attckers:
        worker = Worker(
            idx=idx,
            processor=processor,
            model=local_model,
            data_set=train_subset,
            device=device,
            class_names=class_names,
            rounds=rounds,
            round_to_start_attack=round_to_start_attack,  # epoch_to_start_attack
            epochs=local_epochs,
            attack_type=attack_type,
            attack_params=attack_params,
            batch_size=batch_size,
        )
    else:
        worker = Worker(
            idx=idx,
            processor=processor,
            model=local_model,
            data_set=train_subset,
            device=device,
            class_names=class_names,
            rounds=rounds,
            round_to_start_attack=rounds + 1,
            epochs=local_epochs,
            batch_size=batch_size,
        )
    workers.append(worker)

# create the aggregator
aggregator = Aggregator(
    class_names=class_names,
    processor=processor,
    target_model=target_model,
    device=device,
)



csv_file = os.path.join(output_dir, "aggregator_summary.csv")
aggregator_summary = []
target_model_params = aggregator.get_trainable_params()

for round in range(rounds):
    gradients = {}
    for idx, worker in enumerate(workers):
        worker.set_trainable_params(target_model_params)
        gradient = worker.train()
        gradients[idx] = gradient

    flattend_gradients = []
    for idx, gradient in gradients.items():
        flattend_gradients.append(
            torch.cat([params.flatten() for params in gradient.values()])
        )

    gradients_tensor = torch.stack(flattend_gradients)
    # gradients_tensor = torch.stack(gradients)
    normalized_gradients = gradients_tensor / gradients_tensor.norm(dim=1, keepdim=True)
    cosine_similarity_matrix = (
        torch.mm(normalized_gradients, normalized_gradients.T).to("cpu").numpy()
    )

    # heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(cosine_similarity_matrix, annot=False, cmap="Reds", cbar=True)
    plt.title(f"Cosine Similarity Matrix - round {round}")
    plt.xlabel("Worker")
    plt.ylabel("Worker")
    plt.savefig(os.path.join(output_dir, f"cosine_similarity_round_{round}.png"))
    plt.close()

    # print the shape of gradients tensor
    print(f"Gradients tensor shape: {gradients_tensor.shape}")
    # save the gradients tensor
    torch.save(
        gradients_tensor, os.path.join(output_dir, f"gradients_tensor_round_{round}.pt")
    )

    # t-SNE
    # tsne = TSNE(n_components=2, random_state=200, perplexity=5)
    # grandients_tsne = tsne.fit_transform(gradients_tensor[:, -20481:-1].cpu().numpy()) # only the last layer
    # plt.figure(figsize=(10, 8))
    # plt.scatter(grandients_tsne[:, 0], grandients_tsne[:, 1])
    # plt.title(f"t-SNE - round {round}")
    # plt.xlabel("t-SNE 1")
    # plt.ylabel("t-SNE 2")
    # plt.savefig(os.path.join(output_dir, f"tsne_round_{round}.png"))
    # plt.close()

    gradients_tensor_copy = gradients_tensor.clone()
    # DCT transform
    gradients_numpy = gradients_tensor_copy.cpu().numpy()
    gradients_dct = dct(gradients_numpy, axis=1, norm="ortho")
    gradients_tensor_dct = torch.tensor(gradients_dct, dtype=torch.float32).to(device)
    # cosine similarity matrix after DCT
    normalized_gradients_dct = gradients_tensor_dct / gradients_tensor_dct.norm(
        dim=1, keepdim=True
    )
    cosine_similarity_matrix_dct = (
        torch.mm(normalized_gradients_dct, normalized_gradients_dct.T).to("cpu").numpy()
    )
    # heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(cosine_similarity_matrix_dct, annot=False, cmap="Reds", cbar=True)
    plt.title(f"Cosine Similarity Matrix after DCT - round {round + 1}")
    plt.xlabel("Worker")
    plt.ylabel("Worker")
    plt.savefig(os.path.join(output_dir, f"cosine_similarity_dct_round_{round}.png"))
    plt.close()

    # t-SNE after DCT
    # tsne = TSNE(n_components=2, random_state=200, perplexity=5)
    # gradients_tsne_dct = tsne.fit_transform(gradients_tensor_dct.cpu().numpy())
    # plt.figure(figsize=(10, 8))
    # plt.scatter(gradients_tsne_dct[:, 0], gradients_tsne_dct[:, 1])
    # plt.title(f"t-SNE after DCT - round {round}")
    # plt.xlabel("t-SNE 1")
    # plt.ylabel("t-SNE 2")
    # plt.savefig(os.path.join(output_dir, f"tsne_dct_round_{round}.png"))
    # plt.close()

    mean_gradients = gradients_tensor.mean(dim=0)

    start = 0
    for name, param in target_model_params.items():
        end = start + param.numel()
        target_model_params[name] = mean_gradients[start:end].reshape(param.shape) + param
        start = end
    assert start == mean_gradients.numel(), "The number of parameters does not match"

    aggregator.set_trainable_params(target_model_params)
    normal_accuracy, normal_loss = aggregator.eval(test_loader)
    print(
        f"Round {round + 1} -Normal Accuracy: {normal_accuracy:.4f}, Loss: {normal_loss:.4f}"
    )
    backdoor_accuracy, backdoor_loss = aggregator.eval(data_backdoor_loader)
    print(
        f"Round {round + 1} -Backdoor Accuracy: {backdoor_accuracy:.4f}, Loss: {backdoor_loss:.4f}"
    )
    aggregator_summary.append(
        {
            "normal_accuracy": normal_accuracy,
            "normal_loss": normal_loss,
            "backdoor_accuracy": backdoor_accuracy,
            "backdoor_loss": backdoor_loss,
        }
    )

# save the summary of aggregator
import pandas as pd
df = pd.DataFrame(aggregator_summary)
df.to_csv(csv_file, index=False)

# save the summary of worker
for idx, worker in workers:
    csv_file = os.path.join(output_dir, f"woker{idx}_summary.csv")

# save the model
import datetime
save_path = f"{output_dir}/models_cifar100"
os.makedirs(save_path)

aggregator.save(f"{save_path}/aggregator.pt")

for idx, worker in enumerate(workers):
    worker.save(f"{save_path}/worker_{idx}.pt")


In [ ]:
from src import plot_loss_curve, plot_accuracy_curve

for idx, worker in enumerate(workers):
    fig, axs = plt.subplots(2, 1, figsize=(12, 8))
    train_loader_length = len(worker.data_loader)
    worker.to(torch.device("cpu"))
    loss_values = [summary["loss"] for summary in worker.train_summaries]
    accuracy_values = [summary["accuracy"] for summary in worker.train_summaries]

    plot_loss_curve(axs[0], loss_values)
    plot_accuracy_curve(axs[1], accuracy_values)

    fig.suptitle(f"Worker {idx} Training Summary")
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

normal_accuracy = [summary["normal_accuracy"] for summary in aggregator_summary]
normal_loss = [summary["normal_loss"] for summary in aggregator_summary]
backdoor_accuracy = [summary["backdoor_accuracy"] for summary in aggregator_summary]
backdoor_loss = [summary["backdoor_loss"] for summary in aggregator_summary]

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel("Epoch")
ax1.set_ylabel("Accuracy", color="tab:blue")
ax1.plot(normal_accuracy, label="Normal Accuracy", marker="o", color="tab:blue")
ax1.plot(backdoor_accuracy, label="Backdoor Accuracy", marker="o", color="tab:cyan")
ax1.tick_params(axis="y", labelcolor="tab:blue")

ax2 = ax1.twinx()
ax2.set_ylabel("Loss", color="tab:red")
ax2.plot(normal_loss, label="Normal Loss", marker="o", color="tab:red")
ax2.plot(backdoor_loss, label="Backdoor Loss", marker="o", color="tab:orange")
ax2.tick_params(axis="y", labelcolor="tab:red")

ax1.legend(loc="upper left")
ax2.legend(loc="upper right")

plt.title("Training Metrics Over Time")

plt.show()

# Save the model with current time
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.mkdir(current_time)
aggregator.save(f"{current_time}/aggregator.pt")

for idx, worker in enumerate(workers):
    worker.save(f"{current_time}/worker_{idx}.pt")

In [ ]:
# Evaluate the backdoor attack success rate by one of the attacker's train dataset
aggregator.to(device)
for idx, worker in enumerate(workers):
    print(f"Accuracy on workers[{idx}].dataset: {aggregator.eval(worker.data_loader)}")

print(f"Accuracy on test dataset: {aggregator.eval(test_loader)}")
aggregator.to(torch.device("cpu"))

In [ ]:
# Evaluate the backdoor attack success rate by sampling from the target class in one of the attacker's train dataset
data_set = workers[0].data_set
original_data_set = data_set.dataset

indices = data_set.indices
print(f"Original dataset size: {len(original_data_set)}")

target_label = attack_params["target_label"]
print(f"Target label: {target_label}")

filtered_indices = [i for i in indices if original_data_set.targets[i] == target_label]
print(f"Filtered dataset size: {len(filtered_indices)}")

filtered_data_set = torch.utils.data.Subset(original_data_set, filtered_indices)
data_loader = DataLoader(filtered_data_set, batch_size=128, shuffle=False)
print(f"data_loader size: {len(data_loader)}")

aggregator.to(device)
print(f"Backdoor attack success rate: {aggregator.eval(data_loader)}")
aggregator.to(torch.device("cpu"))

In [ ]:
cifar100_test = CIFAR100(
    os.path.expanduser("~\Desktop\clip_lora\.cache"),
    train=False,
    transform=lambda image: processor(images=image, return_tensors="pt")[
        "pixel_values"
    ].squeeze(0),
    download=False,
)


# put the trigger into the test dataset
for i in range(len(cifar100_test)):
    image = cifar100_test.data[i]
    x, y = attack_params["trigger_position"]
    h, w = attack_params["trigger_size"]
    image[x : x + h, y : y + w, :] = attack_params["trigger_value"]
    cifar100_test.data[i] = image
    cifar100_test.targets[i] = attack_params["target_label"]

data_loader = DataLoader(cifar100_test, batch_size=128, shuffle=False)
print(f"Test dataset size: {len(cifar100_test)}")
aggregator.to(device)
print(f"Backdoor attack success rate on test dataset: {aggregator.eval(data_loader)}")
aggregator.to(torch.device("cpu"))

In [ ]:
import random
from PIL import Image

cifar100_test = CIFAR100(
    os.path.expanduser("~\Desktop\clip_lora\.cache"),
    train=False,
    transform=lambda image: processor(images=image, return_tensors="pt")[
        "pixel_values"
    ].squeeze(0),
    download=False,
)
idx = random.randint(0, len(cifar100_test))

original_image, label = cifar100_test.data[idx], cifar100_test[idx][1]
# draw the image
plt.imshow(Image.fromarray(original_image))
plt.title(f"Original Label: {class_names[label]}")
plt.axis("off")
plt.show()

image = processor(images=original_image, return_tensors="pt")["pixel_values"].squeeze(0)
predicetion = aggregator.predict(image.unsqueeze(0))
print(f"Predicted Label Before Backdoor: {class_names[predicetion]}")

x, y = attack_params["trigger_position"]
h, w = attack_params["trigger_size"]
original_image[x : x + 1, y : y + 1, :] = attack_params["trigger_value"]
# draw the image
plt.imshow(Image.fromarray(original_image))
plt.axis("off")
plt.show()

image = processor(images=original_image, return_tensors="pt")["pixel_values"].squeeze(0)
plt.imshow(image.permute(1, 2, 0))
plt.axis("off")
plt.show()
predicetion = aggregator.predict(image.unsqueeze(0))
print(f"Predicted Label After Backdoor: {class_names[predicetion]}")

In [ ]:
# # backdoor on CIFAR-10 with 10 workers and 3 attackers
# import os
# import torch
# from torch.utils.data import random_split, DataLoader
# from torchvision.datasets import CIFAR10
# from torchvision import transforms
# from transformers import CLIPProcessor
# from peft import LoraConfig
# import matplotlib.pyplot as plt
# import seaborn as sns

# from src import Worker
# from src import Aggregator


# # Load CIFAR-10 dataset
# os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
# cache_dir = os.path.expanduser("~/Desktop/clip_lora/.cache/models")

# output_dir = "cosine_similarity_backdoor_cifar10"
# os.makedirs(output_dir, exist_ok=True)

# # load the processor
# processor = CLIPProcessor.from_pretrained(
#     "openai/clip-vit-base-patch32", cache_dir=cache_dir
# )
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# cifar10_train = CIFAR10(
#     os.path.expanduser("~\Desktop\clip_lora\.cache"),
#     train=True,
#     transform=lambda image: processor(images=image, return_tensors="pt")["pixel_values"].squeeze(0),
#     download=True,
# )
# cifar10_test = CIFAR10(
#     os.path.expanduser("~\Desktop\clip_lora\.cache"),
#     train=False,
#     transform=lambda image: processor(images=image, return_tensors="pt")["pixel_values"].squeeze(0),
#     download=True,
# )

# class_names = cifar10_train.classes
# train_size = len(cifar10_train)
# test_size = len(cifar10_test)
# test_loader = DataLoader(cifar10_test, batch_size=256, shuffle=False)

# # config for federated learning
# num_workers = 10
# rounds = 15
# round_to_start_attack = 5
# local_epochs = 10
# # Create workers for each subset
# workers = []
# attckers = [0, 1, 2]

# attack_type = "backdoor"
# attack_params={"trigger_position": (0, 0), "trigger_size": (15, 15), "backdoor_rate": 0.7, "target_label": 6, "trigger_value": 0}

# # Split the train dataset into {num_workers} subsets
# subset_size = train_size // num_workers
# remaining = train_size % num_workers
# print(f"subset_size: {subset_size}, remaining: {remaining}")
# subset_lengths = [
#     subset_size + 1 if i < remaining else subset_size for i in range(num_workers)
# ]
# train_subsets = random_split(cifar10_train, subset_lengths)

# print(f"Train dataset size: {len(cifar10_train)}")
# print(f"Test dataset size: {len(cifar10_test)}")

# # Define the LoraConfig
# config = LoraConfig(
#     r=16,
#     lora_alpha=16,
#     target_modules=["k_proj", "v_proj", "q_proj", "out_proj", "fc1", "fc2", "visual_projection", "text_projection"],
#     lora_dropout=0.1,
#     bias="none",
# )

# for idx, train_subset in enumerate(train_subsets):
#     if idx in attckers:
#         worker = Worker(
#             idx = idx,
#             data_set = train_subset,
#             lora_config=config,
#             class_names=class_names,
#             cache_dir=cache_dir,
#             rounds=rounds,
#             round_to_start_attack=round_to_start_attack, # epoch_to_start_attack
#             epochs=local_epochs,
#             attack_type=attack_type,
#             attack_params=attack_params,
#         )
#     else:
#         worker = Worker(
#             idx = idx,
#             data_set = train_subset,
#             lora_config=config,
#             class_names=class_names,
#             cache_dir=cache_dir,
#             rounds=rounds,
#             round_to_start_attack= rounds + 1,
#             epochs=local_epochs,
#         )
#     workers.append(worker)

# aggregator = Aggregator(config, class_names, cache_dir)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# aggregator_accuracy_summary = []
# aggregator_loss_summary = []
# for round in range(rounds):
#     trainable_params_vector = aggregator.get_trainable_params_vector()
#     for worker in workers:
#         worker.set_trainable_params_vector(trainable_params_vector)
#     gradients = []
#     for idx, worker in enumerate(workers):
#         worker.to(device)
#         gradients.append(worker.train().to(torch.device("cpu")))
#         worker.to(torch.device("cpu"))

#     gradients_tensor = torch.stack(gradients)
#     normalized_gradients = gradients_tensor / gradients_tensor.norm(dim=1, keepdim=True)
#     cosine_similarity_matrix = torch.mm(normalized_gradients, normalized_gradients.T)

#     # heatmap
#     plt.figure(figsize=(10, 8))
#     sns.heatmap(cosine_similarity_matrix, annot=True, fmt=".2f", cmap="Reds", cbar=True)
#     plt.title(f"Cosine Similarity Matrix - round {round + 1}")
#     plt.xlabel("Worker")
#     plt.ylabel("Worker")
#     plt.savefig(os.path.join(output_dir, f"cosine_similarity_round_{round + 1}.png"))
#     plt.close()
#     mean_gradients = torch.stack(gradients).mean(dim=0)
#     trainable_params_vector = trainable_params_vector + mean_gradients
#     aggregator.set_trainable_params_vector(trainable_params_vector)
#     aggregator.to(device)
#     accuracy, loss = aggregator.eval(test_loader)
#     aggregator.to(torch.device("cpu"))
#     aggregator_accuracy_summary.append(accuracy)
#     aggregator_loss_summary.append(loss)

In [ ]:
# from src import plot_loss_curve, plot_accuracy_curve

# for idx, worker in enumerate(workers):
#     fig, axs = plt.subplots(1, 2, figsize=(12, 6))
#     train_loader_length = len(worker.data_loader)
#     worker.to(torch.device("cpu"))
#     loss_values = [summary["loss"] for summary in worker.train_summaries]
#     accuracy_values = [summary["accuracy"] for summary in worker.train_summaries]

#     plot_loss_curve(axs[0], loss_values)
#     plot_accuracy_curve(axs[1], accuracy_values)

#     fig.suptitle(f"Worker {idx} Training Summary")
#     plt.tight_layout(rect=[0, 0, 1, 0.96])
#     plt.show()

In [ ]:
# fig, axs = plt.subplots(2, 1, figsize=(12, 10))
# axs[0].plot(aggregator_accuracy_summary, label='Accuracy', color='blue', marker='o')
# axs[0].set_title('Aggregator Accuracy Curve')
# axs[0].set_xlabel('Round')
# axs[0].set_ylabel('Accuracy')
# axs[0].grid(True)
# axs[0].legend()

# axs[1].plot(aggregator_loss_summary, label='Loss', color='red', marker='o')
# axs[1].set_title('Aggregator Loss Curve')
# axs[1].set_xlabel('Round')
# axs[1].set_ylabel('Loss')
# axs[1].grid(True)
# axs[1].legend()

# fig.suptitle("Aggregator Performance Over Rounds")
# plt.tight_layout(rect=[0, 0, 1, 0.96])
# plt.show()

# # Save the model with current time
# import datetime

# current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# os.mkdir(current_time)
# aggregator.save(f"{current_time}/aggregator.pt")

# for idx, worker in enumerate(workers):
#     worker.save(f"{current_time}/worker_{idx}.pt")

In [ ]:
# # Evaluate the model on the test dataset
# print(f"Normal accuracy: {aggregator.eval(test_loader)}")

# # Evaluate the backdoor attack success rate by one of the attacker's train dataset
# print(f"Backdoor attack success rate: {aggregator.eval(workers[0].data_loader)}")